## Step 1 - Climate Analysis and Exploration

* Choose a start date and end date for your trip. Make sure that your vacation range is approximately 3-15 days total.

In [ ]:
# import libraries
%matplotlib notebook
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime 
from dateutil import relativedelta

#### Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper.
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# Create a connection to a SQLite database.
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# Reflect an existing database into a new model
# the Base class reflect the database tables.
Base = automap_base() 
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
# view as dictionaries.
Base.classes.keys()

In [ ]:
# Save references to each table.
Climate_measure = Base.classes.measurement
Climate_station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB.
session = Session(engine)

In [ ]:
# Visualization of how the data is stored in measurement dict...
first_row = session.query(Climate_measure).first()
first_row.__dict__

In [ ]:
# And station dict.
sec_row = session.query(Climate_station).first()
sec_row.__dict__

## Precipitation Analysis

### Exploratory Climate Analysis

* Design a query to retrieve the last 12 months of precipitation data and plot the results.

In [ ]:
# To get the last 12 months, we sort the date values,
# get the first one found in the database and substract twelve months.
last_date = session.query(Climate_measure.date).order_by(Climate_measure.date.desc()).first()
# Changing the date format using 'strftime' and 'relativedelta'
last_date_form = datetime.datetime.strptime(last_date[0],'%Y-%m-%d') # Last date stored
months_ago = last_date_form - relativedelta.relativedelta(years=1) # Date 12 months ago
months_ago_form = months_ago.strftime('%Y-%m-%d') 

In [ ]:
# Selecting only the 'date' and 'prcp' values.
prcp_date = session.query(Climate_measure.date, Climate_measure.prcp).\
    order_by(Climate_measure.date.desc()).\
    group_by(Climate_measure.date).\
    filter(Climate_measure.date >= months_ago_form).all()

prcp_date

In [ ]:
# Load the query results into a Pandas DataFrame and set the index to the date column.
# And dropping the None values.
df_pcrp = pd.DataFrame(prcp_date, columns=['Date','Precipitation']).dropna()
#df_pcrp

In [ ]:
# Sort the dataframe by date.
df_pcrp_sort = df_pcrp.sort_values(by=['Date']).set_index('Date')
df_pcrp_sort

In [ ]:
# New dataframe with non_zero values
df_nonzeros = df_pcrp_sort[df_pcrp_sort['Precipitation'] > 0.0]

# Use Pandas Plotting with Matplotlib to plot the data.
ax = df_nonzeros.plot.bar(title= "Data Precipitation per Date")
plt.ylabel('Inches')
plt.xticks(rotation = 'vertical', fontsize=3, ha='right')
plt.tight_layout()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data.
df_pcrp_sort.describe()

## Station Analysis

### Design a query to calculate the total number of stations.

* Design a query to find the most active stations.

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations in descending order.
stations_num = [Climate_measure.station, func.count(Climate_measure.station)]
count_station = session.query(*stations_num).\
    group_by(Climate_measure.station).\
    order_by(func.count(Climate_measure.station).desc()).all()
count_station

In [ ]:
# Getting the station with more info
high_station = count_station[0][0]
high_station

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station
max_temp = session.query(Climate_measure.station, func.max(Climate_measure.tobs)).\
            group_by(Climate_measure.station).\
            filter(Climate_measure.station == high_station).all()

min_temp = session.query(Climate_measure.station, func.min(Climate_measure.tobs)).\
            group_by(Climate_measure.station).\
            filter(Climate_measure.station == high_station).all()

avg_temp = session.query(Climate_measure.station, func.avg(Climate_measure.tobs)).\
            group_by(Climate_measure.id).\
            filter(Climate_measure.station == high_station).all()

# Display information
print(f"Minimum temperature calculated was {min_temp[0][1]} from station {min_temp[0][0]}.")
print(f"Maximum temperature calculated was {max_temp[0][1]} from station {max_temp[0][0]}.")
print(f"Average temperature was {avg_temp[0][1]} from station {avg_temp[0][0]}.")

* Query the last 12 months of temperature observation data for this station and plot the results as a histogram.

In [ ]:
# Design a query to retrieve the last 12 months of temperature observation data (TOBS).
# Filter by the station with the highest number of observations.
high_temps = session.query(Climate_measure.station, Climate_measure.date, Climate_measure.tobs).\
    order_by(Climate_measure.date.desc()).\
    filter(Climate_measure.station == high_station).all()
high_temps

In [ ]:
# Getting the last 12 months and changing the date format
temps_date_form = datetime.datetime.strptime(high_temps[0][1],'%Y-%m-%d')
temps_months_ago = temps_date_form - relativedelta.relativedelta(years=1)
temps_ago_form = temps_months_ago.strftime('%Y-%m-%d')

In [ ]:
# Getting the data points from last 12 months
temps_date = session.query(Climate_measure.date, Climate_measure.tobs).\
    order_by(Climate_measure.date.desc()).\
    filter(Climate_measure.date >= temps_ago_form).\
    filter(Climate_measure.station == high_station).all()
temps_date

In [ ]:
# Plot the results as a histogram with bins=12
# First Pandas DF
df_temps_date = pd.DataFrame(temps_date, columns=['Date', 'Temperature']).set_index('Date')
df_temps_date

In [ ]:
# Plotting histogram
df_temps_date.plot.hist(bins=12,legend=False, title='Temperature Distribution of Station ' + high_station, color='darkblue')
plt.xlabel('Temperature')
plt.tight_layout()